In [47]:
import os
import shutil
import numpy as np
import yaml
import pandas as pd
import sys
import re

## Reading config and file names

In [43]:
os.chdir((sys.path[0]))

In [36]:
with open("../configs/config.yaml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)
    print("Read successful")
print(config)

Read successful
[{'data': {'raw': '/mnt/scratch/drugs/raws', 'fastp': '/mnt/scratch/drugs/fastp', 'fastp_json': '/mnt/scratch/drugs/multiqc/fastp_json', 'referece': '/mnt/scratch/drugs/reference', 'star': '/mnt/scratch/drugs/star'}}, {'star': {'threads': 28}}]


In [37]:
raw = config[0]['data']['raw']
fastp = config[0]['data']['fastp']
fastp_json = config[0]['data']['fastp_json']

In [53]:
metadata = pd.read_csv("../metadata/drugs_metadata.tsv", sep="\t")
mapping = dict(zip(metadata.iloc[:, 1], metadata.iloc[:, 0]))
mapping = {
    re.sub(r'\.fastq(?:\.gz)?$|\.fq(?:\.gz)?$', '', k): v
    for k, v in mapping.items()
}
with open("../configs/multiqc_config.yaml", "w") as f:
    yaml.dump({"sample_names_replace": mapping}, f, default_flow_style=False)

In [22]:
os.chdir(raw)
files = [f for f in os.listdir('.') if os.path.isfile(f)]
files.sort()
len(files)

60

In [41]:
ids = np.unique([
    "_".join(f.split("_")[2:5])
    for f in files
])
len(ids)

30

## Perform quality control with fastp

In [ ]:
for i in range(len(ids)):
    path = "".join([fastp, '/',ids[i],"_fastp"])
    r1 = files[i*2]
    r2 = files[i*2+1]
    out = "".join([path, "/", ids[i]])
    print(out)
    
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)

    os.system(f"fastp -i {r1} -I {r2} -h {out}.html -j {out}.json -o {out}\_R1\_fastp.fq.gz -O {out}\_R2\_fastp.fq.gz -w 16")

## Assemble results with multiqc

In [10]:
os.chdir(fastp)
dirs = os.listdir(".")
len(dirs)

30

In [12]:
os.system(f"find . -name *.json -exec cp {{}} {fastp_json} \;")

0

In [13]:
os.chdir(fastp_json)
os.system("for filename in *.json; do mv $filename ${filename%.*}_fastp.json; done;")

0

In [14]:
files = os.listdir(".")
len(files)

30

In [54]:
os.system(f"multiqc {fastp_json} --filename multiqc_report_fastp -o {fastp_json}/.. -f --interactive -c ../configs/multiqc_config.yaml")


  /// ]8;id=5498;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.14

|           multiqc | MultiQC Version v1.30 now available!
|           multiqc | Search path : /mnt/scratch/drugs/multiqc/fastp_json
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 30/30  
|             fastp | Found 30 reports
|           multiqc | Compressing plot data
|           multiqc | Deleting    : ../../../../../mnt/scratch/drugs/multiqc/multiqc_report_fastp.html   (-f was specified)
|           multiqc | Deleting    : ../../../../../mnt/scratch/drugs/multiqc/multiqc_report_fastp_data   (-f was specified)
|           multiqc | Report      : ../../../../../mnt/scratch/drugs/multiqc/multiqc_report_fastp.html
|           multiqc | Data        : ../../../../../mnt/scratch/drugs/multiqc/multiqc_report_fastp_data
|           multiqc | MultiQC complete


0